In [2]:
import pandas as pd
from datetime import datetime, timedelta

In [6]:
initial_data = pd.read_csv('input.csv')
initial_df = pd.DataFrame(initial_data)

In [9]:
# function to extract compensation and last pay raise date
def extract_compensation(row):
    for col in ['Compensation 2', 'Compensation 1', 'Compensation']:
        if row[col] != '-':
            return row[col]

def extract_last_pay_raise_date(row):
    for col in ['Compensation 2 date', 'Compensation 1 Date', 'Date of Joining']:
        if row[col] != '-':
            return row[col]

In [10]:
historical_analysis_df = initial_df.copy()
historical_analysis_df['Last Compensation'] = historical_analysis_df.apply(extract_compensation, axis=1)
historical_analysis_df['Last Pay Raise Date'] = historical_analysis_df.apply(extract_last_pay_raise_date, axis=1)


In [11]:
historical_analysis_df['Variable Pay'] = ''
historical_analysis_df['Tenure in Org'] = ''
historical_analysis_df['Performance Rating'] = historical_analysis_df['Review 2']
historical_analysis_df['Engagement Score'] = historical_analysis_df['Engagement 2']
historical_analysis_df['Effective Date'] = historical_analysis_df['Date of Joining']
historical_analysis_df['End Date'] = historical_analysis_df.groupby('Employee Code')['Effective Date'].shift(-1) - pd.Timedelta(days=1)
historical_analysis_df.loc[historical_analysis_df['End Date'].isnull(), 'End Date'] = '2100-01-01'

In [14]:
historical_analysis_df.drop(columns=['Date of Joining', 'Date of Exit', 'Compensation 1', 'Compensation 1 date', 
                                     'Compensation 2', 'Compensation 2 date', 'Review 1', 'Review 1 date', 'Review 2', 
                                     'Review 2 date', 'Engagement 1', 'Engagement 1 date', 'Engagement 2', 'Engagement 2 date'], 
                            inplace=True)

In [15]:
historical_analysis_df = historical_analysis_df.reindex(historical_analysis_df.index.repeat(3))
historical_analysis_df.reset_index(drop=True, inplace=True)

In [16]:
historical_analysis_df['Manager Employee Code'] = historical_analysis_df['Manager Employee Code'].apply(lambda x: x if x != '-' else 1)

In [17]:
print(historical_analysis_df)


   Employee Code  Manager Employee Code  Compensation  Last Compensation  \
0              1                    NaN         20000                NaN   
1              1                    NaN         20000                NaN   
2              1                    NaN         20000                NaN   
3              2                    1.0         20000            20000.0   
4              2                    1.0         20000            20000.0   
5              2                    1.0         20000            20000.0   
6              3                    1.0         20000            20000.0   
7              3                    1.0         20000            20000.0   
8              3                    1.0         20000            20000.0   

  Last Pay Raise Date Variable Pay Tenure in Org  Performance Rating  \
0                 NaN                                            NaN   
1                 NaN                                            NaN   
2                 NaN  

In [19]:
historical_analysis_df.to_csv("historical_analysis_df.csv", index=False)